# IMPORTS

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd 
import contextily as ctx
from sklearn.preprocessing import LabelEncoder

import nltk, gensim
nltk.download('punkt')
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

## READ DATA

In [ ]:
data = pd.read_csv('AB_NYC_2019.csv')

# PART 1- READ, EXAMINE, AND CLEAN DATA

## DATA EXPLORATION

In [ ]:
data.head(3)

In [ ]:
data.dtypes, data.isnull().sum()

In [ ]:
data.describe()

## FILLING MISSING VALUES

In [ ]:
data[data['reviews_per_month'].isna()]

In [ ]:
# To check if all reviews_per_month null values actually just have no reviews
data[(data['reviews_per_month'].isna()) & (data['number_of_reviews'] != 0)]

### Now we know that for all missing values in **reviews_per_month**, they should just be 0

In [ ]:
data.reviews_per_month.fillna(0, inplace=True)

### Now let us do **last_review**

In [ ]:
data[(data['last_review'].isna()) & (data['number_of_reviews'] != 0)]

In [ ]:
# Also the same - all last_review NaN have never received a review, ever. However, there is no useful way to fill this data out and it will be of very little use for oure future analysis,=. So we will drop this column
data.drop('last_review', axis=1, inplace=True)

### Now lets deal with **host_name** and **name**

In [ ]:
data.host_name.isnull().sum(), data.name.isnull().sum()

We have no way to know what their real names or host anmes are, and the number of these records are few so we will simply delete these records since filling htem in any other way might affect our word clouds in part 5

In [ ]:
data.dropna(inplace=True) # note that since we already filled out the values that we wanted to, this will only delete the union(16, 21) missing values.

### Expecation: No more null values.

In [ ]:
data.isnull().sum()

## OUTLIER DETECTION

Already, we can see some anomlies with the data - min price = 0 max price = 10000, min number of nights = 1250, min availability_365 = 0.

#### price

In [ ]:
sns.distplot(data.price)

In [ ]:
# let us define outliers as anything less than the 1st percentile and anything more than the 99th percentile. Then our range becomes
data.price.quantile(0.01), data.price.quantile(0.99)

In [ ]:
# The upper limit comes out to be 355, however, in part 4 we need to see all prices upto 1000 so let us use that as the upper limit
sns.distplot(data[data['price'] < 800]['price'])

This looks much better. Note that the price density peaks at close to 50-80, and each peak after than can be explained by the fact that people like to enter whole numbers as prices. A price of 150 or 200 sounds better than 156.62.

In [ ]:
clean_data = data[(data['price']>=30) & (data['price'] <= 800)]

#### minimum_nights

In [ ]:
sns.distplot(data.minimum_nights)

In [ ]:
# let us do the same thing again since there are many listings with very high minimum number of nights
data.minimum_nights.quantile(0.01), data.minimum_nights.quantile(0.99)

In [ ]:
clean_data = clean_data[(clean_data['minimum_nights']>0) & (clean_data['minimum_nights']<=45)]

note that we calculate these quantiles based on the larger dataset because after cleaning each feature we have removed some data records hence affecting the means of other data. We do not want to remove any more data than necessary.

In [ ]:
sns.distplot(clean_data.minimum_nights)

This also looks somewhat better than the previous plot. The peak at 30 days can be explined by people who just want to give their home out on amonthly rent basis.

#### availability_365

In [ ]:
sns.distplot(clean_data.availability_365)

Why so many people with 0 availability? -- Maybe becaus they are inactive/suspended/banned accounts. I believe that there are far too many records with 0 availability that, unfortuately, we will not be able to remove them. Se let us just let them remain.

## FEATURE ENGINEERING
### Some extra features created as needed for practical purposes

In [ ]:
# 1. log_price - we introduce this to see its affect on corelation between different variables
clean_data['log_price'] = np.log10(np.log10(clean_data['price']))

--------------

# PART 2 - EXAMINE PRICE CHANGES

### Let us start by first finding top 5 and bottom 5 neighbourhoods by price, using only neighbourhoods that have more than 5 listings.

In [ ]:
# start by selecting only those neighbourhoods that have occured more than 5 times, first get all counts and then select only those rows which have neighbourhoods that have occured more than 5 times.
neighbourhood_counts = clean_data.neighbourhood.value_counts()
neighbourhood_data = clean_data[clean_data.neighbourhood.isin(neighbourhood_counts.index[neighbourhood_counts.gt(5)])]

In [ ]:
# examine our new data
neighbourhood_data.head()

In [ ]:
neighbourhood_data.groupby(['neighbourhood_group'], as_index=False).agg({'price':['mean']}).sort_values(by=('price', 'mean'))

In [ ]:
neighbourhood_group_data = neighbourhood_data.groupby(['neighbourhood_group', 'neighbourhood'], as_index=False).agg({'price':['mean']})

### Now clearly, top 5 and bottom 5 neighbourhoods are: 
### Bottom: Hunts Point, Tremont, Soundview, Bronxdale, Concord
### Top: Tribeca, NoHp, Flatiron District, Midtown, West Village
### And the boroughs are in the order: Bronx > Staten Island > Queens  > Brooklyn > Manhattan
### Let us plot some of these to see trends
### Let us divide the data by neighbourhood groups

In [ ]:
# plotted to prove a point - this graph is not a good graph
plt.figure(figsize=(15, 15))
plt.xticks(rotation=90)
plt.bar(neighbourhood_group_data['neighbourhood'], neighbourhood_group_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

In [ ]:
# clearly the above graph is mess, we will instead group by borough and plot prices for each neighbourhood
bronx_data = neighbourhood_group_data[neighbourhood_group_data['neighbourhood_group'] == 'Bronx']
queens_data = neighbourhood_group_data[neighbourhood_group_data['neighbourhood_group'] == 'Queens']
statenI_data = neighbourhood_group_data[neighbourhood_group_data['neighbourhood_group'] == 'Staten Island']
brooklyn_data = neighbourhood_group_data[neighbourhood_group_data['neighbourhood_group'] == 'Brooklyn']
manhattan_data = neighbourhood_group_data[neighbourhood_group_data['neighbourhood_group'] == 'Manhattan']

In [ ]:
plt.figure(figsize=(16,5))
plt.bar(neighbourhood_group_data['neighbourhood_group'], neighbourhood_group_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

In [ ]:
plt.figure(figsize=(16,5))
plt.xticks(rotation=90)
plt.bar(bronx_data['neighbourhood'], bronx_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

Observations: Highest mean price: Riverdale, lowest mean price: Hunt's point

In [ ]:
# This looks much better, let us now do the same for other boroughs: 
plt.figure(figsize=(16,5))
plt.xticks(rotation=90)
plt.bar(statenI_data['neighbourhood'], statenI_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

Observations: Very few neighbourhoods, highest mean price: Grymes Hill, lowest mean price: Concord

In [ ]:
plt.figure(figsize=(16,5))
plt.xticks(rotation=90)
plt.bar(queens_data['neighbourhood'], queens_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

Observations: Very high density of neighbourhoods, highest mean price: Belle Harbor, lowest mean price: Corona

In [ ]:
plt.figure(figsize=(16,5))
plt.xticks(rotation=90)
plt.bar(brooklyn_data['neighbourhood'], brooklyn_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

Observations: highest mean price: DUMBO, lowest mean price: Borough park

In [ ]:
plt.figure(figsize=(16,5))
plt.xticks(rotation=90)
plt.bar(manhattan_data['neighbourhood'], manhattan_data[('price', 'mean')])
plt.xlabel('borough')
plt.ylabel('mean price')

Observations: highest mean price: Tribeca, lower mean price: Inwood

-----------------------

# PART 3

### Features selected: log_price, longitude, minimum_nights, number_of_reviews, reviews_per_month, calculated_host_listings_count, and availability_365. These features have been selected asa they are quite diverse and any correlation between any 2 variables will be quite interesting.

In [ ]:

hm = sns.heatmap(clean_data[['log_price', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']].corr(), annot=True)
hm.set(xlabel = 'features', ylabel = 'features')

Aside from the obvious correlation between number of reviews and reviews per month, which is unintersting, we have the following interesting correlation:
- log_price and longitude

- minimum_nights and calculated_host_listings_count - people that buy houses particularly for airbnb and other renting, dont live in that house hence increaing minimum_nights and buy a lot of properties - hence increasing their calculated_host_listings_count

- reviews_per_month and availability_365 - They could be given out mroe often hence increasing their reviews

- availability_365 and minimum_nights - This might be explained by the fact the people that dont live in that plot probably want people to stay for long - sort of like a rent system

We should keep in mind that for such a  large dataset, even small correlations (>=|0.2|) is also significant.

# PART 4

In [ ]:
# let us make a geopandas dataframe to make use of latitude and longitude
geodata = gpd.GeoDataFrame(clean_data[['latitude', 'longitude', 'neighbourhood_group']], geometry = gpd.points_from_xy(clean_data.longitude, clean_data.latitude))

In [ ]:
# examine the data
geodata.head()

In [ ]:
# this is a built in map ot nyc
nyc = gpd.read_file(gpd.datasets.get_path('nybb')) #geopandas file

In [ ]:
# this is the shape of nyc for reference for our later plot
nyc.plot(figsize=(10,10),color='gray', alpha=0.5, edgecolor='black') # plot to work with

In [ ]:
# cool plot :) -- not necessary  PLEASE NOTE - THIS CELL MAY NEED TO BE RUN TWICE DUE TOA BUG IN GEOPANDAS PLOTTING
nyc_plot = nyc.plot(figsize=(10,10),color='gray', alpha=0.5, edgecolor='black') # plot to work with
nyc = nyc.to_crs(epsg=3857)
ctx.add_basemap(nyc_plot)

In [ ]:
geodata.plot(figsize=(15, 15), color='red', markersize=3)

In [ ]:
# use a label encoder to go from bonrx -> 0, Staten Island -> 1, etc
encoder = LabelEncoder()
geodata['neighbourhood_group'] = encoder.fit_transform(geodata['neighbourhood_group'])

In [ ]:

colors = geodata['neighbourhood_group'].unique()
rgb_values = sns.color_palette('Set1', 8)
color_map = dict(zip(colors, rgb_values))
plt.figure(figsize=(15,15), dpi=150)
# plt.figure()
plt.scatter(geodata['longitude'], geodata['latitude'], c=geodata['neighbourhood_group'].map(color_map))

# PART 5

In [ ]:
# generate features from name
name_list = list(clean_data['name'])

In [ ]:
text = ""
for name in name_list:
    tokens = nltk.word_tokenize(name) # tokenize each record
    cleaned_tokens = [token for token in tokens if token.isalnum()] # only take al-num, not !!, .., etc
    text += " ".join(cleaned_tokens)

In [ ]:
wordcloud = WordCloud(max_font_size=60, max_words=200, background_color='white').generate(text)

In [ ]:
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.figure(figsize=(15,15), dpi=200)
plt.show()

# PART 6

In [ ]:
# let us start by examining host_id
len(clean_data['host_id'].unique())

In [ ]:
host_data = clean_data.groupby(['host_id']).agg({'host_id':['count'], 'price': ['mean', 'max', 'min'], 'availability_365': ['mean'], 'number_of_reviews': ['mean']}).sort_values(by=('host_id','count'))

In [ ]:
host_data # note that host_id is the index

In [ ]:
# let us see some of these hosts:
clean_data[clean_data['host_id'] == 219517861]

According to the data, this host in the financial district has the most number of listings. Observations: It looks like these listings are more houses up for rent, considering that they are all up for 29 days minimum. 

It also looks like these houses are up for 327/365 days suggesting that the other days may be spent either living in them or maintaining these houses at different times of the year.

In [ ]:
hm = sns.heatmap(host_data.corr(), annot=True)
hm.set(xlabel = 'features', ylabel = 'features')

The above heatmap clearly shows a positive correlation between availability_365 and number_of_reviews - the more often the listing is avialable -> the more often people stay in it -> the more often they review it. 

Hence the busiest hosts are those that have most plots available for the most ammount of time

Note: the prices are correlated for obvious reasons - they are derived from the same variable

# PART 7

For our first plot, let us observe the availability in the different boroughs dependin on room type

In [ ]:
plt.figure(figsize = (16,10))
sns.boxplot(x='neighbourhood_group', y='availability_365', hue='room_type', data=clean_data)

This plot is clearly interesting because it shows-
- shared rooms are available far more frequently than any other type of room. This makes sense because it does not require the host to leave or have a separate home.
- Staten ISland has a disproportionately small number of shared rooms while having more availability in their private rooms as well as home/apt. This could have a socio-economic backing with people only have secondary houses in Staten Island and prefering to lend them out as it is closer to the beach.

For our second plot let us observe the difference removing the most common words will make on our word cloud.

In [ ]:
# generate features from name
name_list = list(clean_data['name'])

In [ ]:
text = []
for name in name_list:
    tokens = nltk.word_tokenize(name) # tokenize each record
    cleaned_tokens = [token for token in tokens if token.isalnum()] # only take al-num, not !!, .., etc
    text.append(cleaned_tokens)

In [ ]:
corpus_dictionary = gensim.corpora.Dictionary(text)
corpus_dictionary.filter_extremes(no_below=10, no_above=0.5) # filter out words that have occured less than 5 times and filter out top 50% of the most common words

In [ ]:
corpus_dict = dict(corpus_dictionary)

In [ ]:
all_text = ""
for k,v in corpus_dict.items():
     all_text += " " + v

In [ ]:
wordcloud = WordCloud(max_font_size=60, max_words=100, background_color='white').generate(all_text)

In [ ]:
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.figure(figsize=(15,15), dpi=200)
plt.show()

The word cloud seems to have become more about the stay thant he place of stay. In the previous word cloud we could see things like Brooklyn, Manhattan, etc. Now we don't. This tells us that most of the host_name lisitngs had the place of stay in them and since we removed to top 50% of the vocabulary, we don't see those words anymore.